In [66]:
import random
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np
import warnings
from tqdm import tqdm_notebook as tqdm
import requests
import json
import API
import matplotlib.pyplot as plt
import GoogleDistanceMatrix
import GoogleVRP
import routing
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
warnings.filterwarnings('ignore')

In [67]:
order = pd.read_excel('../1. Sales Data/DMS Data (Jun-Sep2020)/DMS order_Jun-Sep2020 (Only store with DMS available).xlsx')
order = order.loc[order['Instant Order']==True]
store = pd.read_csv('Data/StoreLocation.csv')

In [68]:
order['orderTime'] = pd.to_datetime(order['下單時間'])
order['arrivalTime'] = pd.to_datetime(order['訂單送達時間'])
order['departureTime'] = pd.to_datetime(order['取餐出發時間'])

order['waitingTime'] = order['arrivalTime'] - order['orderTime']
order['tripDurationReal'] = pd.to_datetime(order['訂單送達時間']) - pd.to_datetime(order['departureTime'])
order['tripDurationReal'] = order['tripDurationReal'].dt.seconds

In [69]:
pd.to_datetime('2020-06-29')

Timestamp('2020-06-29 00:00:00')

In [70]:
order = order.loc[(order['orderTime']>=pd.to_datetime('2020-06-29'))&(order['orderTime']<=pd.to_datetime('2020-07-05'))]

In [71]:
order = order.loc[order['門店名稱'].isin(['PH64', 'PH57', 'PHD629', 'PHD606'])]

In [72]:
order.shape

(1329, 19)

In [73]:
def get_lat_lon(x):
    location = x['配送地址']
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address=香港'+location+'&key='+API.key
    data = requests.get(url)
    text = json.loads(data.text)
    if text['results'] == []:
        return('missing')
    lat = str(text['results'][0]['geometry']['location']['lat'])
    lng = str(text['results'][0]['geometry']['location']['lng'])
    return(lat+','+lng)

In [74]:
order['DeliveryLatLon'] = order.apply(get_lat_lon,axis=1)

In [75]:

def tripDurationFromGoogle(x):
    # distance matrix API
    StoreLatLon,DeliveryLatLon = x['StoreLatLon'],x['DeliveryLatLon']
    url = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&origins='+StoreLatLon+\
    '&destinations='+DeliveryLatLon+'&key='+API.key
    data = requests.get(url)
    text = json.loads(data.text)
    if text['rows'] != []:
        if 'duration' in text['rows'][0]['elements'][0]:
            
            duration = text['rows'][0]['elements'][0]['duration']['value']
            return duration
    else:
        return 'missing'

In [76]:
order = order.sort_values(by=['取餐出發時間','Rider'])
order['last取餐出發時間'] = order['取餐出發時間'].shift(1)
order['lastRider'] = order['Rider'].shift(1)
order['pre取餐出發時間'] = order['取餐出發時間'].shift(-1)
order['preRider'] = order['Rider'].shift(-1)

In [77]:
order['shared'] = 0
order.loc[((order['last取餐出發時間']==order['取餐出發時間'])&(order['lastRider']==order['Rider']))|
                   ((order['pre取餐出發時間']==order['取餐出發時間'])&(order['preRider']==order['Rider'])),'shared'] = 1

In [78]:
order = order[['日期', '門店名稱', '配送地址', '下單時間', '指派成功時間', '騎手接單時間', '到店取餐時間', '取餐出發時間',
       '訂單送達時間', '預送達時間', 'Banner', 'Instant Order', '收貨地址坐標', 'Rider',
       'orderTime', 'arrivalTime', 'waitingTime',
       'last取餐出發時間', 'lastRider', 'departureTime', 'tripDurationReal',
       'DeliveryLatLon','shared']]

In [79]:
order = order.merge(store, how='left',left_on='門店名稱', right_on='storeCode')
order['StoreLatLon'] = order['lat'].astype(str)+','+order['lon'].astype(str)

In [80]:
order['tripDurationFromGoogle'] = order.apply(lambda x:tripDurationFromGoogle(x),axis=1)

In [81]:
order['tripDurationFromGoogle'].fillna('missing',inplace=True)
order = order.loc[order['tripDurationFromGoogle']!='missing']
order['tripDurationFromGoogle'] = order['tripDurationFromGoogle'].astype(int)

In [82]:
order.loc[order['shared']==0]['tripDurationFromGoogle'].mean()/60

7.563275193798449

In [83]:
order.loc[order['shared']==0]['tripDurationReal'].mean()/60

20.135364583333335

In [84]:
timeParameter = (order.loc[order['shared']==0]['tripDurationReal']/ order.loc[order['shared']==0]['tripDurationFromGoogle']).median()
order['timeParameter'] = timeParameter
timeParameter

2.135738592635864

In [85]:
order['foodPreparationTime'] = pd.to_datetime(order['取餐出發時間']) - pd.to_datetime(order['下單時間'])

In [86]:
order.loc[order['shared']==0]['foodPreparationTime'].mean()

Timedelta('0 days 00:28:43.434375')

In [87]:
order.loc[order['shared']==1]['foodPreparationTime'].mean()

Timedelta('0 days 00:33:41.250788')

In [88]:
order = order.loc[order['tripDurationReal']>order['tripDurationFromGoogle']]

In [89]:
order.to_csv('Data/ProcessedJun29.csv',index=False)